In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sys

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'
# Load training datasets and testing datasets 
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5420/2861151304.py:20: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5420/2861151304.py:22: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')


In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcom, ycom = preprocessing(test_complete)

In [4]:
def evaluate_detection(model, x, y): 
    # Need to return FNR, F1, Precision, Recall, Accuracy, number of false positives, number of false negatives, number of true positives, number of true negatives
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y, axis=1)
    report = classification_report(y_true, y_pred, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    accuracy = report['accuracy']
    FNR = fn / (fn + tp)

    
    return FNR, f1, precision, recall, accuracy, cm

In [8]:
corr3c5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr53Cavg.hdf5')
corr3c25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr253Cavg.hdf5')
corr3c50 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr503Cavg.hdf5')
corr3g5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr53Gavg.hdf5')
corr3g25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr253Gavg.hdf5')
corr3g50 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr503Gavg.hdf5')

In [6]:
evaluate_detection(corr3c5, xbasic, ybasic)

(0.010248618784530386,
 0.9180986447905306,
 0.9267800594450063,
 0.918840749072424,
 0.918840749072424,
 array([[54146, 10370],
        [  742, 71658]], dtype=int64))

In [7]:
evaluate_detection(corr3c5, xplus, yplus)

(0.023938323993391856,
 0.7449847862228615,
 0.8119110396538817,
 0.7634418368088353,
 0.7634418368088353,
 array([[11897, 12284],
        [  739, 30132]], dtype=int64))

In [8]:
evaluate_detection(corr3c25, xbasic, ybasic)

(0.01255524861878453,
 0.9839582869405543,
 0.983970662963597,
 0.9839609687691724,
 0.9839609687691724,
 array([[63229,  1287],
        [  909, 71491]], dtype=int64))

In [9]:
evaluate_detection(corr3c25, xplus, yplus)

(0.011402286935959315,
 0.7884540368927155,
 0.844334649351584,
 0.8010426505849015,
 0.8010426505849015,
 array([[13580, 10601],
        [  352, 30519]], dtype=int64))

In [10]:
evaluate_detection(corr3c50, xbasic, ybasic)

(0.010276243093922652,
 0.9779549154317135,
 0.9781830540609326,
 0.9779718951766047,
 0.9779718951766047,
 array([[62244,  2272],
        [  744, 71656]], dtype=int64))

In [11]:
evaluate_detection(corr3c50, xplus, yplus)

(0.01120792977227819,
 0.7853341121432225,
 0.8429088245423307,
 0.7984269418004796,
 0.7984269418004796,
 array([[13430, 10751],
        [  346, 30525]], dtype=int64))

In [12]:
evaluate_detection(corr3g5, xbasic, ybasic)

(0.013232044198895027,
 0.9735519267354799,
 0.9738316774014399,
 0.973575038709866,
 0.973575038709866,
 array([[61856,  2660],
        [  958, 71442]], dtype=int64))

In [13]:
evaluate_detection(corr3g5, xplus, yplus)

(0.012406465614978459,
 0.8205135437845434,
 0.8608133572667405,
 0.8286710746203589,
 0.8286710746203589,
 array([[15132,  9049],
        [  383, 30488]], dtype=int64))

In [14]:
evaluate_detection(corr3g25, xbasic, ybasic)

(0.014958563535911603,
 0.9864313300484122,
 0.986443035436232,
 0.9864296356890356,
 0.9864296356890356,
 array([[63741,   775],
        [ 1083, 71317]], dtype=int64))

In [15]:
evaluate_detection(corr3g25, xplus, yplus)

(0.01334585857277056,
 0.8643057549560927,
 0.886921073906556,
 0.8681973406960691,
 0.8681973406960691,
 array([[17337,  6844],
        [  412, 30459]], dtype=int64))

In [16]:
evaluate_detection(corr3g50, xbasic, ybasic)

(0.014806629834254143,
 0.9889740519129036,
 0.9890192719638692,
 0.988971340091735,
 0.988971340091735,
 array([[64078,   438],
        [ 1072, 71328]], dtype=int64))

In [17]:
evaluate_detection(corr3g50, xplus, yplus)

(0.012665608499886625,
 0.8636963260637445,
 0.886847151134282,
 0.8676705660103176,
 0.8676705660103176,
 array([[17287,  6894],
        [  391, 30480]], dtype=int64))

#### AVG 

In [10]:
corr5B5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bavg.hdf5')
corr5B10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Bavg.hdf5')
corr5B25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Bavg.hdf5')
corr5C15 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1avg.hdf5')
corr5C110 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105C1avg.hdf5')
corr5C125 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255C1avg.hdf5')
corr5C35 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55C3avg.hdf5')
corr5C310 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105C3avg.hdf5')
corr5C325 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3avg.hdf5')

In [11]:
evaluate_detection(corr5B5, xcom, ycom)

(0.005021971123666039,
 0.9254076174924102,
 0.9340827185788555,
 0.9257649840621055,
 0.9257649840621055,
 array([[61401, 10283],
        [  360, 71325]], dtype=int64))

In [12]:
evaluate_detection(corr5B10, xcom, ycom)

(0.004422124572783706,
 0.9148313587498618,
 0.9263479035262767,
 0.9153791963395155,
 0.9153791963395155,
 array([[59869, 11815],
        [  317, 71368]], dtype=int64))

In [13]:
evaluate_detection(corr5B25, xcom, ycom)

(0.005803166631791867,
 0.9292708896736208,
 0.9368656256740572,
 0.9295663637187955,
 0.9295663637187955,
 array([[62002,  9682],
        [  416, 71269]], dtype=int64))

In [14]:
evaluate_detection(corr5C15, xcom, ycom)

(0.046997279765641345,
 0.975860822454523,
 0.9768712293801273,
 0.9758734454449707,
 0.9758734454449707,
 array([[71594,    90],
        [ 3369, 68316]], dtype=int64))

In [15]:
evaluate_detection(corr5C110, xcom, ycom)

(0.01887424147311153,
 0.9900249482572724,
 0.9901810518164235,
 0.9900257377815288,
 0.9900257377815288,
 array([[71607,    77],
        [ 1353, 70332]], dtype=int64))

In [16]:
evaluate_detection(corr5C125, xcom, ycom)

(0.038962125967775686,
 0.9804626010679018,
 0.9811968136566175,
 0.9804699760757207,
 0.9804699760757207,
 array([[71677,     7],
        [ 2793, 68892]], dtype=int64))

In [17]:
evaluate_detection(corr5C35, xcom, ycom)

(0.015498360884424914,
 0.9600442159177495,
 0.9611693239843311,
 0.9600680760833932,
 0.9600680760833932,
 array([[67070,  4614],
        [ 1111, 70574]], dtype=int64))

In [18]:
evaluate_detection(corr5C310, xcom, ycom)

(0.015484410964636954,
 0.984055127037257,
 0.9840555406944596,
 0.9840551304675348,
 0.9840551304675348,
 array([[70508,  1176],
        [ 1110, 70575]], dtype=int64))

In [19]:
evaluate_detection(corr5C325, xcom, ycom)

(0.01648880518937016,
 0.983008874841105,
 0.9830093663372428,
 0.983008879185877,
 0.983008879185877,
 array([[70430,  1254],
        [ 1182, 70503]], dtype=int64))

In [7]:
evaluate_detection(corr5B5, xbasic, ybasic)

(0.004558011049723757,
 0.9560772557703201,
 0.9588618004852771,
 0.9562432440328377,
 0.9562432440328377,
 array([[58855,  5661],
        [  330, 72070]], dtype=int64))

In [8]:
evaluate_detection(corr5B5, xplus, yplus)

(0.01454439441547083,
 0.8822572409757126,
 0.8985446903073606,
 0.8848906488410957,
 0.8848906488410957,
 array([[18293,  5888],
        [  449, 30422]], dtype=int64))

In [9]:
evaluate_detection(corr5B10, xbasic, ybasic)

(0.004129834254143646,
 0.948103720135536,
 0.9521770208609084,
 0.9483624996348126,
 0.9483624996348126,
 array([[57745,  6771],
        [  299, 72101]], dtype=int64))

In [10]:
evaluate_detection(corr5B10, xplus, yplus)

(0.012827572802954228,
 0.8604375955006761,
 0.8846596668569583,
 0.8646552350504977,
 0.8646552350504977,
 array([[17126,  7055],
        [  396, 30475]], dtype=int64))

In [11]:
evaluate_detection(corr5B25, xbasic, ybasic)

(0.0052900552486187845,
 0.9564431931758209,
 0.9590661290639726,
 0.956601127698735,
 0.956601127698735,
 array([[58957,  5559],
        [  383, 72017]], dtype=int64))

In [12]:
evaluate_detection(corr5B25, xplus, yplus)

(0.01250364419681902,
 0.8673570867825495,
 0.8893179846143305,
 0.8710673545011989,
 0.8710673545011989,
 array([[17469,  6712],
        [  386, 30485]], dtype=int64))

In [13]:
evaluate_detection(corr5C15, xbasic, ybasic)

(0.0626243093922652,
 0.9668897656867086,
 0.9690345154576933,
 0.9668628940372199,
 0.9668628940372199,
 array([[64513,     3],
        [ 4534, 67866]], dtype=int64))

In [14]:
evaluate_detection(corr5C15, xplus, yplus)

(0.1865828771338797,
 0.8955566319421113,
 0.9154998776044775,
 0.8953716486231199,
 0.8953716486231199,
 array([[24181,     0],
        [ 5760, 25111]], dtype=int64))

In [15]:
evaluate_detection(corr5C110, xbasic, ybasic)

(0.01888121546961326,
 0.9899986226483048,
 0.9902001605974614,
 0.9899938648514418,
 0.9899938648514418,
 array([[64513,     3],
        [ 1367, 71033]], dtype=int64))

In [16]:
evaluate_detection(corr5C110, xplus, yplus)

(0.027501538660879142,
 0.9845855778790374,
 0.9850820400334382,
 0.9845600523141756,
 0.9845600523141756,
 array([[24180,     1],
        [  849, 30022]], dtype=int64))

In [17]:
evaluate_detection(corr5C125, xbasic, ybasic)

(0.04825966850828729,
 0.9744505128900312,
 0.9757355470526499,
 0.9744295772590493,
 0.9744295772590493,
 array([[64509,     7],
        [ 3494, 68906]], dtype=int64))

In [18]:
evaluate_detection(corr5C125, xplus, yplus)

(0.17731851899841275,
 0.9007852290323695,
 0.9189210677814361,
 0.9005667369032915,
 0.9005667369032915,
 array([[24181,     0],
        [ 5474, 25397]], dtype=int64))

In [19]:
evaluate_detection(corr5C35, xbasic, ybasic)

(0.015455801104972376,
 0.9900774749113563,
 0.9901625178749895,
 0.9900742060825616,
 0.9900742060825616,
 array([[64276,   240],
        [ 1119, 71281]], dtype=int64))

In [20]:
evaluate_detection(corr5C35, xplus, yplus)

(0.017135823264552492,
 0.9469688561753138,
 0.9488898335585798,
 0.9472498728474896,
 0.9472498728474896,
 array([[21806,  2375],
        [  529, 30342]], dtype=int64))

In [21]:
evaluate_detection(corr5C310, xbasic, ybasic)

(0.015469613259668509,
 0.9902162674857102,
 0.9903052955349512,
 0.9902129772999503,
 0.9902129772999503,
 array([[64296,   220],
        [ 1120, 71280]], dtype=int64))

In [22]:
evaluate_detection(corr5C310, xplus, yplus)

(0.017103430403938972,
 0.9476694014296102,
 0.9495162369660378,
 0.9479401293322677,
 0.9479401293322677,
 array([[21843,  2338],
        [  528, 30343]], dtype=int64))

In [23]:
evaluate_detection(corr5C325, xbasic, ybasic)

(0.01680939226519337,
 0.9890627905095888,
 0.9891592171717783,
 0.9890589850711385,
 0.9890589850711385,
 array([[64235,   281],
        [ 1217, 71183]], dtype=int64))

In [24]:
evaluate_detection(corr5C325, xplus, yplus)

(0.01694146610087137,
 0.9472986746263102,
 0.949208839513966,
 0.9475768364455424,
 0.9475768364455424,
 array([[21818,  2363],
        [  523, 30348]], dtype=int64))

#### TRIM

In [20]:
corr5B5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Btrim.hdf5')
corr5B10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Btrim.hdf5')
corr5B25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Btrim.hdf5')
corr5C15 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1trim.hdf5')
#corr5C110 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105C1trim.hdf5')
#corr5C125 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255C1trim.hdf5')
#corr5C35 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55C3trim.hdf5')
#corr5C310 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105C3trim.hdf5')
#corr5C325 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3trim.hdf5')

In [21]:
evaluate_detection(corr5B5, xcom, ycom)

(0.004994071284090116,
 0.929065449720116,
 0.9368995449049262,
 0.9293710634795528,
 0.9293710634795528,
 array([[61916,  9768],
        [  358, 71327]], dtype=int64))

In [22]:
evaluate_detection(corr5B10, xcom, ycom)

(0.004380274813419823,
 0.9181372028786184,
 0.9287909551669837,
 0.9186225753126548,
 0.9186225753126548,
 array([[60331, 11353],
        [  314, 71371]], dtype=int64))

In [23]:
evaluate_detection(corr5B25, xcom, ycom)

(0.004422124572783706,
 0.9142192863544645,
 0.9259010807163767,
 0.9147793456046983,
 0.9147793456046983,
 array([[59783, 11901],
        [  317, 71368]], dtype=int64))

In [24]:
evaluate_detection(corr5C15, xcom, ycom)

(0.027411592383343796,
 0.9856360124014779,
 0.9859695194240652,
 0.9856384574071103,
 0.9856384574071103,
 array([[71590,    94],
        [ 1965, 69720]], dtype=int64))

In [ ]:
evaluate_detection(corr5C110, xcom, ycom)

(0.01887424147311153,
 0.9900249482572724,
 0.9901810518164235,
 0.9900257377815288,
 0.9900257377815288,
 array([[71607,    77],
        [ 1353, 70332]], dtype=int64))

In [ ]:
evaluate_detection(corr5C125, xcom, ycom)

(0.038962125967775686,
 0.9804626010679018,
 0.9811968136566175,
 0.9804699760757207,
 0.9804699760757207,
 array([[71677,     7],
        [ 2793, 68892]], dtype=int64))

In [ ]:
evaluate_detection(corr5C35, xcom, ycom)

(0.015498360884424914,
 0.9600442159177495,
 0.9611693239843311,
 0.9600680760833932,
 0.9600680760833932,
 array([[67070,  4614],
        [ 1111, 70574]], dtype=int64))

In [ ]:
evaluate_detection(corr5C310, xcom, ycom)

(0.015484410964636954,
 0.984055127037257,
 0.9840555406944596,
 0.9840551304675348,
 0.9840551304675348,
 array([[70508,  1176],
        [ 1110, 70575]], dtype=int64))

In [ ]:
evaluate_detection(corr5C325, xcom, ycom)

(0.01648880518937016,
 0.983008874841105,
 0.9830093663372428,
 0.983008879185877,
 0.983008879185877,
 array([[70430,  1254],
        [ 1182, 70503]], dtype=int64))

In [27]:
evaluate_detection(corr5B5, xbasic, ybasic)

(0.004392265193370166,
 0.959231231785833,
 0.9616180683113731,
 0.9593692482982267,
 0.9593692482982267,
 array([[59271,  5245],
        [  318, 72082]], dtype=int64))

In [28]:
evaluate_detection(corr5B5, xplus, yplus)

(0.013443037154611124,
 0.8594891071087266,
 0.8837468987190035,
 0.8637470028336846,
 0.8637470028336846,
 array([[17095,  7086],
        [  415, 30456]], dtype=int64))

In [29]:
evaluate_detection(corr5B10, xbasic, ybasic)

(0.0038674033149171273,
 0.9508598994241823,
 0.9545341635122602,
 0.9510867977446025,
 0.9510867977446025,
 array([[58099,  6417],
        [  280, 72120]], dtype=int64))

In [30]:
evaluate_detection(corr5B10, xplus, yplus)

(0.012568429918046062,
 0.8312715046375951,
 0.8668976370039558,
 0.8382075128968974,
 0.8382075128968974,
 array([[15662,  8519],
        [  388, 30483]], dtype=int64))

In [31]:
evaluate_detection(corr5B25, xbasic, ybasic)

(0.003908839779005525,
 0.9465919401131463,
 0.9509623857296089,
 0.9468725349849543,
 0.9468725349849543,
 array([[57525,  6991],
        [  283, 72117]], dtype=int64))

In [32]:
evaluate_detection(corr5B25, xplus, yplus)

(0.012633215639273105,
 0.8716070442680398,
 0.8920751366213381,
 0.8750090823221681,
 0.8750090823221681,
 array([[17690,  6491],
        [  390, 30481]], dtype=int64))

In [11]:
evaluate_detection(corr5C15, xbasic, ybasic)

(0.03363259668508287,
 0.9822060764648011,
 0.9828387569128129,
 0.9821934616845365,
 0.9821934616845365,
 array([[64513,     3],
        [ 2435, 69965]], dtype=int64))

In [12]:
evaluate_detection(corr5C15, xplus, yplus)

(0.086521330698714,
 0.9507851310990553,
 0.9552772476117818,
 0.9506103320496985,
 0.9506103320496985,
 array([[24133,    48],
        [ 2671, 28200]], dtype=int64))

In [ ]:
evaluate_detection(corr5C110, xbasic, ybasic)

(0.01888121546961326,
 0.9899986226483048,
 0.9902001605974614,
 0.9899938648514418,
 0.9899938648514418,
 array([[64513,     3],
        [ 1367, 71033]], dtype=int64))

In [ ]:
evaluate_detection(corr5C110, xplus, yplus)

(0.027501538660879142,
 0.9845855778790374,
 0.9850820400334382,
 0.9845600523141756,
 0.9845600523141756,
 array([[24180,     1],
        [  849, 30022]], dtype=int64))

In [ ]:
evaluate_detection(corr5C125, xbasic, ybasic)

(0.04825966850828729,
 0.9744505128900312,
 0.9757355470526499,
 0.9744295772590493,
 0.9744295772590493,
 array([[64509,     7],
        [ 3494, 68906]], dtype=int64))

In [ ]:
evaluate_detection(corr5C125, xplus, yplus)

(0.17731851899841275,
 0.9007852290323695,
 0.9189210677814361,
 0.9005667369032915,
 0.9005667369032915,
 array([[24181,     0],
        [ 5474, 25397]], dtype=int64))

In [ ]:
evaluate_detection(corr5C35, xbasic, ybasic)

(0.015455801104972376,
 0.9900774749113563,
 0.9901625178749895,
 0.9900742060825616,
 0.9900742060825616,
 array([[64276,   240],
        [ 1119, 71281]], dtype=int64))

In [ ]:
evaluate_detection(corr5C35, xplus, yplus)

(0.017135823264552492,
 0.9469688561753138,
 0.9488898335585798,
 0.9472498728474896,
 0.9472498728474896,
 array([[21806,  2375],
        [  529, 30342]], dtype=int64))

In [ ]:
evaluate_detection(corr5C310, xbasic, ybasic)

(0.015469613259668509,
 0.9902162674857102,
 0.9903052955349512,
 0.9902129772999503,
 0.9902129772999503,
 array([[64296,   220],
        [ 1120, 71280]], dtype=int64))

In [ ]:
evaluate_detection(corr5C310, xplus, yplus)

(0.017103430403938972,
 0.9476694014296102,
 0.9495162369660378,
 0.9479401293322677,
 0.9479401293322677,
 array([[21843,  2338],
        [  528, 30343]], dtype=int64))

In [ ]:
evaluate_detection(corr5C325, xbasic, ybasic)

(0.01680939226519337,
 0.9890627905095888,
 0.9891592171717783,
 0.9890589850711385,
 0.9890589850711385,
 array([[64235,   281],
        [ 1217, 71183]], dtype=int64))

In [ ]:
evaluate_detection(corr5C325, xplus, yplus)

(0.01694146610087137,
 0.9472986746263102,
 0.949208839513966,
 0.9475768364455424,
 0.9475768364455424,
 array([[21818,  2363],
        [  523, 30348]], dtype=int64))

#### med

In [25]:
corr5B5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bmed.hdf5')
corr5B10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Bmed.hdf5')
corr5B25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Bmed.hdf5')
corr5C15 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1med.hdf5')

In [26]:
evaluate_detection(corr5B5, xcom, ycom)

(0.005370719118365069,
 0.9272726131216085,
 0.9354248101334217,
 0.9275994113092788,
 0.9275994113092788,
 array([[61689,  9995],
        [  385, 71300]], dtype=int64))

In [27]:
evaluate_detection(corr5B10, xcom, ycom)

(0.006193764385854781,
 0.9276908345918159,
 0.9355474760885154,
 0.9280039618048532,
 0.9280039618048532,
 array([[61806,  9878],
        [  444, 71241]], dtype=int64))

In [28]:
evaluate_detection(corr5B25, xcom, ycom)

(0.004784822487270698,
 0.9050719095943358,
 0.9192282410348668,
 0.9058304096422518,
 0.9058304096422518,
 array([[58526, 13158],
        [  343, 71342]], dtype=int64))

In [29]:
evaluate_detection(corr5C15, xcom, ycom)

(0.018372044360744924,
 0.9907922158545586,
 0.9909579520824148,
 0.9907929887214112,
 0.9907929887214112,
 array([[71681,     3],
        [ 1317, 70368]], dtype=int64))

In [ ]:
evaluate_detection(corr5C110, xcom, ycom)

(0.01887424147311153,
 0.9900249482572724,
 0.9901810518164235,
 0.9900257377815288,
 0.9900257377815288,
 array([[71607,    77],
        [ 1353, 70332]], dtype=int64))

In [ ]:
evaluate_detection(corr5C125, xcom, ycom)

(0.038962125967775686,
 0.9804626010679018,
 0.9811968136566175,
 0.9804699760757207,
 0.9804699760757207,
 array([[71677,     7],
        [ 2793, 68892]], dtype=int64))

In [ ]:
evaluate_detection(corr5C35, xcom, ycom)

(0.015498360884424914,
 0.9600442159177495,
 0.9611693239843311,
 0.9600680760833932,
 0.9600680760833932,
 array([[67070,  4614],
        [ 1111, 70574]], dtype=int64))

In [ ]:
evaluate_detection(corr5C310, xcom, ycom)

(0.015484410964636954,
 0.984055127037257,
 0.9840555406944596,
 0.9840551304675348,
 0.9840551304675348,
 array([[70508,  1176],
        [ 1110, 70575]], dtype=int64))

In [ ]:
evaluate_detection(corr5C325, xcom, ycom)

(0.01648880518937016,
 0.983008874841105,
 0.9830093663372428,
 0.983008879185877,
 0.983008879185877,
 array([[70430,  1254],
        [ 1182, 70503]], dtype=int64))

In [34]:
evaluate_detection(corr5B5, xbasic, ybasic)

(0.004903314917127072,
 0.9570748654616772,
 0.9596707431635976,
 0.9572292500511262,
 0.9572292500511262,
 array([[59015,  5501],
        [  355, 72045]], dtype=int64))

In [35]:
evaluate_detection(corr5B5, xplus, yplus)

(0.015127465906514204,
 0.8974500173269647,
 0.909251160417791,
 0.8992588825110804,
 0.8992588825110804,
 array([[19102,  5079],
        [  467, 30404]], dtype=int64))

In [36]:
evaluate_detection(corr5B10, xbasic, ybasic)

(0.005441988950276243,
 0.9546814595297561,
 0.9575231853740371,
 0.95485553185895,
 0.95485553185895,
 array([[58729,  5787],
        [  394, 72006]], dtype=int64))

In [37]:
evaluate_detection(corr5B10, xplus, yplus)

(0.015516180233876454,
 0.8946788398873966,
 0.9070789639401701,
 0.8966068444379859,
 0.8966068444379859,
 array([[18968,  5213],
        [  479, 30392]], dtype=int64))

In [38]:
evaluate_detection(corr5B25, xbasic, ybasic)

(0.0040607734806629835,
 0.9369931311537052,
 0.9431179109978058,
 0.9374214847059511,
 0.9374214847059511,
 array([[56242,  8274],
        [  294, 72106]], dtype=int64))

In [39]:
evaluate_detection(corr5B25, xplus, yplus)

(0.012017751287616209,
 0.8613149899300455,
 0.8856218871314806,
 0.8655089733343021,
 0.8655089733343021,
 array([[17148,  7033],
        [  371, 30500]], dtype=int64))

In [14]:
evaluate_detection(corr5C15, xbasic, ybasic)

(0.018121546961325966,
 0.9903999907930509,
 0.990585726092373,
 0.9903955710070408,
 0.9903955710070408,
 array([[64513,     3],
        [ 1312, 71088]], dtype=int64))

In [15]:
evaluate_detection(corr5C15, xplus, yplus)

(0.021476466586764276,
 0.9879729119059167,
 0.9882782308608551,
 0.9879568408050571,
 0.9879568408050571,
 array([[24181,     0],
        [  663, 30208]], dtype=int64))

In [ ]:
evaluate_detection(corr5C110, xbasic, ybasic)

(0.01888121546961326,
 0.9899986226483048,
 0.9902001605974614,
 0.9899938648514418,
 0.9899938648514418,
 array([[64513,     3],
        [ 1367, 71033]], dtype=int64))

In [ ]:
evaluate_detection(corr5C110, xplus, yplus)

(0.027501538660879142,
 0.9845855778790374,
 0.9850820400334382,
 0.9845600523141756,
 0.9845600523141756,
 array([[24180,     1],
        [  849, 30022]], dtype=int64))

In [ ]:
evaluate_detection(corr5C125, xbasic, ybasic)

(0.04825966850828729,
 0.9744505128900312,
 0.9757355470526499,
 0.9744295772590493,
 0.9744295772590493,
 array([[64509,     7],
        [ 3494, 68906]], dtype=int64))

In [ ]:
evaluate_detection(corr5C125, xplus, yplus)

(0.17731851899841275,
 0.9007852290323695,
 0.9189210677814361,
 0.9005667369032915,
 0.9005667369032915,
 array([[24181,     0],
        [ 5474, 25397]], dtype=int64))

In [ ]:
evaluate_detection(corr5C35, xbasic, ybasic)

(0.015455801104972376,
 0.9900774749113563,
 0.9901625178749895,
 0.9900742060825616,
 0.9900742060825616,
 array([[64276,   240],
        [ 1119, 71281]], dtype=int64))

In [ ]:
evaluate_detection(corr5C35, xplus, yplus)

(0.017135823264552492,
 0.9469688561753138,
 0.9488898335585798,
 0.9472498728474896,
 0.9472498728474896,
 array([[21806,  2375],
        [  529, 30342]], dtype=int64))

In [ ]:
evaluate_detection(corr5C310, xbasic, ybasic)

(0.015469613259668509,
 0.9902162674857102,
 0.9903052955349512,
 0.9902129772999503,
 0.9902129772999503,
 array([[64296,   220],
        [ 1120, 71280]], dtype=int64))

In [ ]:
evaluate_detection(corr5C310, xplus, yplus)

(0.017103430403938972,
 0.9476694014296102,
 0.9495162369660378,
 0.9479401293322677,
 0.9479401293322677,
 array([[21843,  2338],
        [  528, 30343]], dtype=int64))

In [ ]:
evaluate_detection(corr5C325, xbasic, ybasic)

(0.01680939226519337,
 0.9890627905095888,
 0.9891592171717783,
 0.9890589850711385,
 0.9890589850711385,
 array([[64235,   281],
        [ 1217, 71183]], dtype=int64))

In [ ]:
evaluate_detection(corr5C325, xplus, yplus)

(0.01694146610087137,
 0.9472986746263102,
 0.949208839513966,
 0.9475768364455424,
 0.9475768364455424,
 array([[21818,  2363],
        [  523, 30348]], dtype=int64))

#### krum

In [33]:
corr5b5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bkrum.hdf5')
corr5c15 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1krum.hdf5')

In [35]:
evaluate_detection(corr5B5, xcom, ycom)

(0.005370719118365069,
 0.9272726131216085,
 0.9354248101334217,
 0.9275994113092788,
 0.9275994113092788,
 array([[61689,  9995],
        [  385, 71300]], dtype=int64))

In [ ]:
evaluate_detection(corr5B10, xcom, ycom)

(0.004422124572783706,
 0.9148313587498618,
 0.9263479035262767,
 0.9153791963395155,
 0.9153791963395155,
 array([[59869, 11815],
        [  317, 71368]], dtype=int64))

In [ ]:
evaluate_detection(corr5B25, xcom, ycom)

(0.005803166631791867,
 0.9292708896736208,
 0.9368656256740572,
 0.9295663637187955,
 0.9295663637187955,
 array([[62002,  9682],
        [  416, 71269]], dtype=int64))

In [36]:
evaluate_detection(corr5C15, xcom, ycom)

(0.018372044360744924,
 0.9907922158545586,
 0.9909579520824148,
 0.9907929887214112,
 0.9907929887214112,
 array([[71681,     3],
        [ 1317, 70368]], dtype=int64))

In [ ]:
evaluate_detection(corr5C110, xcom, ycom)

(0.01887424147311153,
 0.9900249482572724,
 0.9901810518164235,
 0.9900257377815288,
 0.9900257377815288,
 array([[71607,    77],
        [ 1353, 70332]], dtype=int64))

In [ ]:
evaluate_detection(corr5C125, xcom, ycom)

(0.038962125967775686,
 0.9804626010679018,
 0.9811968136566175,
 0.9804699760757207,
 0.9804699760757207,
 array([[71677,     7],
        [ 2793, 68892]], dtype=int64))

In [ ]:
evaluate_detection(corr5C35, xcom, ycom)

(0.015498360884424914,
 0.9600442159177495,
 0.9611693239843311,
 0.9600680760833932,
 0.9600680760833932,
 array([[67070,  4614],
        [ 1111, 70574]], dtype=int64))

In [ ]:
evaluate_detection(corr5C310, xcom, ycom)

(0.015484410964636954,
 0.984055127037257,
 0.9840555406944596,
 0.9840551304675348,
 0.9840551304675348,
 array([[70508,  1176],
        [ 1110, 70575]], dtype=int64))

In [ ]:
evaluate_detection(corr5C325, xcom, ycom)

(0.01648880518937016,
 0.983008874841105,
 0.9830093663372428,
 0.983008879185877,
 0.983008879185877,
 array([[70430,  1254],
        [ 1182, 70503]], dtype=int64))

In [8]:
evaluate_detection(corr5b5, xbasic, ybasic)

(0.002679558011049724,
 0.9449435130368761,
 0.9498496010451459,
 0.9452584066142744,
 0.9452584066142744,
 array([[57215,  7301],
        [  194, 72206]], dtype=int64))

In [9]:
evaluate_detection(corr5b5, xplus, yplus)

(0.010689644002461857,
 0.7826309359104554,
 0.8419750922243834,
 0.7962108551914554,
 0.7962108551914554,
 array([[13292, 10889],
        [  330, 30541]], dtype=int64))

In [17]:
evaluate_detection(corr5c15, xbasic, ybasic)

(0.018107734806629833,
 0.9903634698516782,
 0.9905472636848005,
 0.9903590522656227,
 0.9903590522656227,
 array([[64507,     9],
        [ 1311, 71089]], dtype=int64))

In [18]:
evaluate_detection(corr5c15, xplus, yplus)

(0.020601859350199217,
 0.9806997341844538,
 0.9807371103277795,
 0.9806909830705515,
 0.9806909830705515,
 array([[23754,   427],
        [  636, 30235]], dtype=int64))

#### geo

In [37]:
corr5b5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bgeo.hdf5')
corr5c15 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1geo.hdf5')

In [38]:
evaluate_detection(corr5b5, xcom, ycom)

(0.004463974332147591,
 0.9251326689973141,
 0.9340153451376743,
 0.9254999337374188,
 0.9254999337374188,
 array([[61323, 10361],
        [  320, 71365]], dtype=int64))

In [45]:
evaluate_detection(corr5b5, xplus, yplus)

(0.011143144051051148,
 0.8002756404314799,
 0.8506515144349549,
 0.8111603574802005,
 0.8111603574802005,
 array([[14129, 10052],
        [  344, 30527]], dtype=int64))

In [39]:
evaluate_detection(corr5c15, xcom, ycom)

(0.030731673292878566,
 0.9845816153937013,
 0.9850404235586885,
 0.9845852311169081,
 0.9845852311169081,
 array([[71677,     7],
        [ 2203, 69482]], dtype=int64))

In [21]:
evaluate_detection(corr5c15, xplus, yplus)

(0.13922451491691232,
 0.9221756408354781,
 0.9336873622329488,
 0.9219101939984015,
 0.9219101939984015,
 array([[24180,     1],
        [ 4298, 26573]], dtype=int64))

#### Corr 5A

In [6]:
a5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Aavg.hdf5')
a10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Aavg.hdf5')
a25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Aavg.hdf5')
t5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Atrim.hdf5')
t10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Atrim.hdf5')
t25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Atrim.hdf5')
m5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Amed.hdf5')
m10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Amed.hdf5')
m25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Amed.hdf5')
g5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Ageo.hdf5')
g10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Ageo.hdf5')
g25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Ageo.hdf5')
k5 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr55Akrum.hdf5')
k10 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr105Akrum.hdf5')
k25 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idcorr255Akrum.hdf5')

In [8]:
evaluate_detection(a5, xbasic, ybasic)

(0.0048480662983425415,
 0.9703233702758131,
 0.9714401924272534,
 0.9703833007099243,
 0.9703833007099243,
 array([[60812,  3704],
        [  351, 72049]], dtype=int64))

In [9]:
evaluate_detection(a5, xplus, yplus)

(0.010041786790191442,
 0.7792450382246261,
 0.8408396545166887,
 0.7934498292523432,
 0.7934498292523432,
 array([[13120, 11061],
        [  310, 30561]], dtype=int64))

In [10]:
evaluate_detection(a5, xcom, ycom)

(0.005747366952640023,
 0.9439542896745875,
 0.9486096385822101,
 0.9440953065167504,
 0.9440953065167504,
 array([[64081,  7603],
        [  412, 71273]], dtype=int64))

In [11]:
evaluate_detection(a10, xbasic, ybasic)

(0.005552486187845304,
 0.970509245722283,
 0.9715416851376081,
 0.9705658944170148,
 0.9705658944170148,
 array([[60888,  3628],
        [  402, 71998]], dtype=int64))

In [12]:
evaluate_detection(a10, xplus, yplus)

(0.00955589388098863,
 0.7212779894909753,
 0.8151147441726448,
 0.7469846690401802,
 0.7469846690401802,
 array([[10547, 13634],
        [  295, 30576]], dtype=int64))

In [13]:
evaluate_detection(a10, xcom, ycom)

(0.006542512380553812,
 0.945814327597907,
 0.9500014879158338,
 0.9459367087724683,
 0.9459367087724683,
 array([[64402,  7282],
        [  469, 71216]], dtype=int64))

In [14]:
evaluate_detection(a25, xbasic, ybasic)

(0.0032596685082872928,
 0.966386723597014,
 0.9680696941098779,
 0.9664757953781881,
 0.9664757953781881,
 array([[60162,  4354],
        [  236, 72164]], dtype=int64))

In [15]:
evaluate_detection(a25, xplus, yplus)

(0.012665608499886625,
 0.6955668634465182,
 0.8007693349375912,
 0.7269853956259537,
 0.7269853956259537,
 array([[ 9542, 14639],
        [  391, 30480]], dtype=int64))

In [16]:
evaluate_detection(a25, xcom, ycom)

(0.0038362279416893352,
 0.9359552270460666,
 0.9425537261473779,
 0.9361856468274173,
 0.9361856468274173,
 array([[62810,  8874],
        [  275, 71410]], dtype=int64))

In [17]:
evaluate_detection(t5, xbasic, ybasic)

(0.007334254143646409,
 0.9728185377982685,
 0.9735154730730012,
 0.9728592713780713,
 0.9728592713780713,
 array([[61331,  3185],
        [  531, 71869]], dtype=int64))

In [18]:
evaluate_detection(t5, xplus, yplus)

(0.012406465614978459,
 0.8206576736913368,
 0.8608948198651091,
 0.8287982271307128,
 0.8287982271307128,
 array([[15139,  9042],
        [  383, 30488]], dtype=int64))

In [19]:
evaluate_detection(t5, xcom, ycom)

(0.008049103717653624,
 0.9551809612066414,
 0.957708554847883,
 0.9552413701706784,
 0.9552413701706784,
 array([[65844,  5840],
        [  577, 71108]], dtype=int64))

In [20]:
evaluate_detection(t10, xbasic, ybasic)

(0.004654696132596685,
 0.9699702812954014,
 0.9711383790561191,
 0.9700327207923106,
 0.9700327207923106,
 array([[60750,  3766],
        [  337, 72063]], dtype=int64))

In [21]:
evaluate_detection(t10, xplus, yplus)

(0.01078682258430242,
 0.7937612346782192,
 0.8475302701275563,
 0.80562014095764,
 0.80562014095764,
 array([[13813, 10368],
        [  333, 30538]], dtype=int64))

In [22]:
evaluate_detection(t10, xcom, ycom)

(0.005552068075608565,
 0.9428559732809739,
 0.9477463773341135,
 0.9430072051838263,
 0.9430072051838263,
 array([[63911,  7773],
        [  398, 71287]], dtype=int64))

In [23]:
evaluate_detection(t25, xbasic, ybasic)

(0.0027071823204419887,
 0.9664137908651006,
 0.968161154710334,
 0.9665050103713225,
 0.9665050103713225,
 array([[60126,  4390],
        [  196, 72204]], dtype=int64))

In [24]:
evaluate_detection(t25, xplus, yplus)

(0.005118071976936283,
 0.6358010262225898,
 0.7898784531411435,
 0.6865508973334302,
 0.6865508973334302,
 array([[ 7083, 17098],
        [  158, 30713]], dtype=int64))

In [25]:
evaluate_detection(t25, xcom, ycom)

(0.003236381390807003,
 0.9330635225456002,
 0.9404208801608436,
 0.9333328683327637,
 0.9333328683327637,
 array([[62358,  9326],
        [  232, 71453]], dtype=int64))

In [26]:
evaluate_detection(m5, xbasic, ybasic)

(0.0077486187845303865,
 0.9747699338127015,
 0.9753060270434274,
 0.974802068421514,
 0.974802068421514,
 array([[61627,  2889],
        [  561, 71839]], dtype=int64))

In [27]:
evaluate_detection(m5, xplus, yplus)

(0.010851608305529462,
 0.7811387610664655,
 0.841100703673347,
 0.7949393300879168,
 0.7949393300879168,
 array([[13227, 10954],
        [  335, 30536]], dtype=int64))

In [28]:
evaluate_detection(m5, xcom, ycom)

(0.00856525074980819,
 0.9590557282592816,
 0.9610267793958107,
 0.9590985498957236,
 0.9590985498957236,
 array([[66434,  5250],
        [  614, 71071]], dtype=int64))

In [29]:
evaluate_detection(m10, xbasic, ybasic)

(0.007582872928176796,
 0.9742197222524658,
 0.9748025513826907,
 0.9742542873002424,
 0.9742542873002424,
 array([[61540,  2976],
        [  549, 71851]], dtype=int64))

In [30]:
evaluate_detection(m10, xplus, yplus)

(0.012309287033137896,
 0.8677772285101247,
 0.8896848809784373,
 0.8714669766765967,
 0.8714669766765967,
 array([[17485,  6696],
        [  380, 30491]], dtype=int64))

In [31]:
evaluate_detection(m10, xcom, ycom)

(0.008453651391504498,
 0.9586493612877948,
 0.9606827922425386,
 0.9586939994001492,
 0.9586939994001492,
 array([[66368,  5316],
        [  606, 71079]], dtype=int64))

In [32]:
evaluate_detection(m25, xbasic, ybasic)

(0.007044198895027625,
 0.972128241436249,
 0.9728986964229371,
 0.972172719039411,
 0.972172719039411,
 array([[61216,  3300],
        [  510, 71890]], dtype=int64))

In [33]:
evaluate_detection(m25, xplus, yplus)

(0.007644715104790904,
 0.7132048262937432,
 0.8140293191190128,
 0.7411538182082394,
 0.7411538182082394,
 array([[10167, 14014],
        [  236, 30635]], dtype=int64))

In [34]:
evaluate_detection(m25, xcom, ycom)

(0.007714305642742554,
 0.9528526132855153,
 0.9557498489090596,
 0.9529256673339425,
 0.9529256673339425,
 array([[65488,  6196],
        [  553, 71132]], dtype=int64))

In [35]:
evaluate_detection(k5, xbasic, ybasic)

(0.0054005524861878455,
 0.9705968326335274,
 0.971636109762449,
 0.9706535393964183,
 0.9706535393964183,
 array([[60889,  3627],
        [  391, 72009]], dtype=int64))

In [36]:
evaluate_detection(k5, xplus, yplus)

(0.009847429626510317,
 0.7332728705765952,
 0.8198943188648957,
 0.7562486376516748,
 0.7562486376516748,
 array([[11066, 13115],
        [  304, 30567]], dtype=int64))

In [37]:
evaluate_detection(k5, xcom, ycom)

(0.006584362139917695,
 0.9464328404975225,
 0.9505083504985741,
 0.9465505095243741,
 0.9465505095243741,
 array([[64493,  7191],
        [  472, 71213]], dtype=int64))

In [38]:
evaluate_detection(k10, xbasic, ybasic)

(0.005497237569060774,
 0.9714632203775988,
 0.9724199668271004,
 0.9715153816938853,
 0.9715153816938853,
 array([[61014,  3502],
        [  398, 72002]], dtype=int64))

In [39]:
evaluate_detection(k10, xplus, yplus)

(0.012276894172524375,
 0.7895104623190683,
 0.8441920596159352,
 0.8018418949356971,
 0.8018418949356971,
 array([[13651, 10530],
        [  379, 30492]], dtype=int64))

In [40]:
evaluate_detection(k10, xcom, ycom)

(0.006389063262886238,
 0.9469236221182135,
 0.9509511168196567,
 0.9470387601224811,
 0.9470387601224811,
 array([[64549,  7135],
        [  458, 71227]], dtype=int64))

In [41]:
evaluate_detection(k25, xbasic, ybasic)

(0.005994475138121547,
 0.9726827033508688,
 0.973498221066896,
 0.9727278039089661,
 0.9727278039089661,
 array([[61216,  3300],
        [  434, 71966]], dtype=int64))

In [42]:
evaluate_detection(k25, xplus, yplus)

(0.01156425123902692,
 0.8393311537717432,
 0.8722525917485232,
 0.8455096999200755,
 0.8455096999200755,
 array([[16033,  8148],
        [  357, 30514]], dtype=int64))

In [43]:
evaluate_detection(k25, xcom, ycom)

(0.006933110134616726,
 0.9516573367734337,
 0.9548472530292678,
 0.951739915881397,
 0.951739915881397,
 array([[65262,  6422],
        [  497, 71188]], dtype=int64))

In [44]:
evaluate_detection(g5, xbasic, ybasic)

(0.005552486187845304,
 0.9727401086043492,
 0.9735887599953557,
 0.972786233895235,
 0.972786233895235,
 array([[61192,  3324],
        [  402, 71998]], dtype=int64))

In [45]:
evaluate_detection(g5, xplus, yplus)

(0.01541900165203589,
 0.7360250499662067,
 0.8154317890204095,
 0.7575383273995495,
 0.7575383273995495,
 array([[11309, 12872],
        [  476, 30395]], dtype=int64))

In [46]:
evaluate_detection(g5, xcom, ycom)

(0.006082165027551091,
 0.9483268441316042,
 0.9521755718178555,
 0.9484337618313582,
 0.9484337618313582,
 array([[64727,  6957],
        [  436, 71249]], dtype=int64))

In [47]:
evaluate_detection(g10, xbasic, ybasic)

(0.00574585635359116,
 0.971691480828758,
 0.9726070256090559,
 0.9717417978906775,
 0.9717417978906775,
 array([[61063,  3453],
        [  416, 71984]], dtype=int64))

In [48]:
evaluate_detection(g10, xplus, yplus)

(0.011952965566389168,
 0.8248670942131826,
 0.8635768653781645,
 0.8325583085083194,
 0.8325583085083194,
 array([[15332,  8849],
        [  369, 30502]], dtype=int64))

In [49]:
evaluate_detection(g10, xcom, ycom)

(0.006668061658645463,
 0.9490948511865016,
 0.9527218917313103,
 0.9491940377626963,
 0.9491940377626963,
 array([[64878,  6806],
        [  478, 71207]], dtype=int64))

In [50]:
evaluate_detection(g25, xbasic, ybasic)

(0.008370165745856353,
 0.9758194239809572,
 0.9762527430319973,
 0.9758465044260715,
 0.9758465044260715,
 array([[61815,  2701],
        [  606, 71794]], dtype=int64))

In [51]:
evaluate_detection(g25, xplus, yplus)

(0.012568429918046062,
 0.8385634886002252,
 0.8712136934730742,
 0.8447467848579525,
 0.8447467848579525,
 array([[16022,  8159],
        [  388, 30483]], dtype=int64))

In [52]:
evaluate_detection(g25, xcom, ycom)

(0.008927948664295181,
 0.9623737746860572,
 0.9639297445494767,
 0.9624047039457624,
 0.9624047039457624,
 array([[66934,  4750],
        [  640, 71045]], dtype=int64))